In [6]:
import os
import re
from getpass import getpass
with open('./sdr-test-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password: ············


In [7]:
# assuming the ep5g setup is prepared
import json
import math
from loguru import logger
import time
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed


### Uplink measurements

In [10]:
# set session parameters
end_name = "endnode01"
edge_name = "edge"
nt_dev = "adv03"
exp_name = "2023-11-03-t1"
client_ip = "172.16.0.40"
server_ip = "10.70.70.3"
sleep_dur = 10
tripm = "oneway"
total_samples = 1e6
interval = "10" # in ms
offset = "0" # ms
length = 100 # in B
mult = 1 # multiple of packets
duration = 30 # in seconds
total_session_time = interval # in seconds
itnum = 1 #math.ceil(total_session_time/duration)
cl_port = "55500"
nt_sleep = "300" # in ms

In [11]:
# run measurements at end node
command = f"DOWNLINK=false EDGE_NAME={edge_name} END_NAME={end_name} NT_DEV={nt_dev} EXP_NAME={exp_name} SERVER_IP={server_ip} CLIENT_IP={client_ip} ITNUM={itnum} SLEEP_DUR={sleep_dur} TRIPM={tripm} INTERVAL={interval}ms INTERVAL_OFFSET={offset}ms LENGTH={length} MULT={mult} DUR={duration}s CL_PORT={cl_port} NT_SLEEP={nt_sleep}ms /tmp/measure-upload.sh > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="end-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-03 15:32:40.351 | INFO     | __main__:<module>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [12]:
 # make parquet
dev_name = "endnode01"
trip = "uplink"

command = f"python3 /tmp/parquets-from-folders.py /mnt/volume/{exp_name}/results /mnt/volume/{exp_name}/client /mnt/volume/{exp_name}/edge/server /mnt/volume/{exp_name}/networkinfo trip={trip} > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-03 15:36:56.135 | INFO     | __main__:<module>:10 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [13]:
# upload parquet files to containers
command = "for f in /mnt/volume/{}/results/*.parquet; do AUTH_SERVER=testbed.expeca.proj.kth.se AUTH_PROJECT_NAME=sdr-test-project AUTH_USERNAME=gourav AUTH_PASSWORD=gouravexpeca python3 /tmp/upload-files.py $f {}; done".format(exp_name,exp_name)
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-03 15:41:19.388 | INFO     | __main__:<module>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


### Downlink Measurements

In [22]:
# set session parameters
end_name = "endnode01"
edge_name = "edge"
nt_dev = "adv03"
exp_name = "tstmps-DL"
server_ip = "172.16.0.40"
client_ip = "10.70.70.3"
sleep_dur = 60
tripm = "oneway"
total_samples = 1e6
interval = 10000 # in us
length = 100 # in B
mult = 1 # multiple of packets
duration = 1667 # in seconds
total_session_time = interval # in seconds
itnum = 1#math.ceil(total_session_time/duration)
cl_port = "55500"
nt_sleep = "300" # in ms

In [ ]:
 ###### run measurements at edge node #####
command = f"DOWNLINK=true EDGE_NAME={edge_name} END_NAME={end_name} NT_DEV={nt_dev} EXP_NAME={exp_name} SERVER_IP={server_ip} CLIENT_IP={client_ip} ITNUM={itnum} SLEEP_DUR={sleep_dur} TRIPM={tripm} INTERVAL={interval}us LENGTH={length} MULT={mult} DUR={duration}s CL_PORT={cl_port} NT_SLEEP={nt_sleep}ms /tmp/measure-upload.sh > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-09-04 20:52:39.887 | INFO     | __main__:<cell line: 7>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [ ]:
dev_name = "endnode01"
trip = "downlink"
 # make parquet
command = f"python3 /tmp/parquets-from-folders.py /mnt/volume/{exp_name}/results /mnt/volume/{exp_name}/edge/client /mnt/volume/{exp_name}/{dev_name}/server /mnt/volume/{exp_name}/{dev_name}/networkinfo trip={trip} > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-09-05 08:31:14.630 | INFO     | __main__:<cell line: 9>:9 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [ ]:
# upload parquet files to containers
command = "for f in /mnt/volume/{}/results/*.parquet; do AUTH_SERVER=testbed.expeca.proj.kth.se AUTH_PROJECT_NAME=sdr-test-project AUTH_USERNAME=gourav AUTH_PASSWORD=gouravexpeca python3 /tmp/upload-files.py $f {}; done".format(exp_name,exp_name)
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-09-03 18:35:59.316 | INFO     | __main__:<cell line: 7>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
